In [1]:
import pandas as pd
df = pd.read_csv('/content/sample_data/03_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [2]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.9 MB/s eta 0:00:00


In [3]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost

voting_reg = VotingRegressor(
    estimators=[
        ('lin_reg', LinearRegression()),
        ('xgboost_01', xgb.XGBRegressor(max_depth=6, n_estimators=40, random_state=42)),
        ('xgboost_02', xgb.XGBRegressor(max_depth=8, n_estimators=40, random_state=42)),
        ('catboost_01',Catboost(iterations=10, depth=8, learning_rate=1, random_state=42)),
        ('catboost_02',Catboost(iterations=20, depth=8, learning_rate=1, random_state=42)),
    ]
)
voting_reg.fit(X_train, y_train)

0:	learn: 1.0682203	total: 252ms	remaining: 2.27s
1:	learn: 1.0623745	total: 434ms	remaining: 1.74s
2:	learn: 1.0603842	total: 601ms	remaining: 1.4s
3:	learn: 1.0601403	total: 778ms	remaining: 1.17s
4:	learn: 1.0597593	total: 951ms	remaining: 951ms
5:	learn: 1.0592751	total: 1.1s	remaining: 735ms
6:	learn: 1.0590332	total: 1.29s	remaining: 552ms
7:	learn: 1.0587974	total: 1.45s	remaining: 362ms
8:	learn: 1.0586414	total: 1.62s	remaining: 180ms
9:	learn: 1.0584606	total: 1.8s	remaining: 0us
0:	learn: 1.0682203	total: 189ms	remaining: 3.59s
1:	learn: 1.0623745	total: 384ms	remaining: 3.45s
2:	learn: 1.0603842	total: 553ms	remaining: 3.13s
3:	learn: 1.0601403	total: 727ms	remaining: 2.91s
4:	learn: 1.0597593	total: 901ms	remaining: 2.7s
5:	learn: 1.0592751	total: 1.05s	remaining: 2.46s
6:	learn: 1.0590332	total: 1.23s	remaining: 2.28s
7:	learn: 1.0587974	total: 1.4s	remaining: 2.1s
8:	learn: 1.0586414	total: 1.58s	remaining: 1.93s
9:	learn: 1.0584606	total: 1.76s	remaining: 1.76s
10:	lear

VotingRegressor(estimators=[('lin_reg', LinearRegression()),
                            ('xgboost_01',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          inte...
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=8,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=40,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=42, ...)),
                            ('catboost_01',
                             <catboost.core.CatBoostRegressor object at 0x7fca7f73d6f0>),
                            ('catboost_02',
                             <catboost.core.CatBoostRegressor object at 0x7fca7f73d3f0>)])

In [4]:
from sklearn.metrics import root_mean_squared_error
y_pred = voting_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

1.0531092209741797

In [5]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [6]:
y_pred = voting_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

1.06072702154975

In [7]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/03_test.csv')
del test_set['Premium Amount']
y_pred = voting_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('voting_reg_04.csv', index=False)